Распознавание цветов (ромашки, розы, подсолнухи, тюльпаны, одуванчики) на изображениях с помощью предварительно обученной нейронной сети VGG16

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования.

In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam

from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.layers.normalization import BatchNormalization

from tensorflow.python.keras.applications import InceptionV3

In [0]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 32
# Количество изображений для обучения
nb_train_samples = 2565
# Количество изображений для проверки
nb_validation_samples = 550
# Количество изображений для тестирования
nb_test_samples = 550

Загружаем предварительно обученную нейронную сеть

In [0]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# Загружаем предварительно обученную нейронную сеть Inceptionv3
#inception_net = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

Создаем составную нейронную сеть на основе VGG16 или inception

In [115]:
vgg16_net.summary()
#inception_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [0]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
#model.add(inception_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))

Компилируем составную нейронную сеть

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

Создаем генератор изображений
Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [0]:
datagen = ImageDataGenerator(rescale=1./255)

Генератор данных для обучения на основе изображений из каталога

In [119]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 2565 images belonging to 5 classes.


Генератор данных для проверки на основе изображений из каталога

In [120]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 550 images belonging to 5 classes.


Генератор данных для тестирования на основе изображений из каталога

In [121]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 550 images belonging to 5 classes.


Обучаем модель с использованием генераторов
train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [122]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
81/81 [==============================] - 24s 300ms/step - loss: 4.4236 - acc: 0.2246 - val_loss: 1.6147 - val_acc: 0.3055
Epoch 2/10
81/81 [==============================] - 21s 264ms/step - loss: 1.7328 - acc: 0.2655 - val_loss: 1.5261 - val_acc: 0.3691
Epoch 3/10
81/81 [==============================] - 22s 267ms/step - loss: 1.5972 - acc: 0.3142 - val_loss: 1.4719 - val_acc: 0.5073
Epoch 4/10
81/81 [==============================] - 22s 269ms/step - loss: 1.5258 - acc: 0.3676 - val_loss: 1.3964 - val_acc: 0.5309
Epoch 5/10
81/81 [==============================] - 22s 271ms/step - loss: 1.4593 - acc: 0.4316 - val_loss: 1.2841 - val_acc: 0.5855
Epoch 6/10
81/81 [==============================] - 22s 273ms/step - loss: 1.4209 - acc: 0.5232 - val_loss: 1.2309 - val_acc: 0.6364
Epoch 7/10
81/81 [==============================] - 22s 274ms/step - loss: 1.4138 - acc: 0.5540 - val_loss: 1.1467 - val_acc: 0.6182
Epoch 8/10
81/81 [==============================] - 22s 276ms/step - 

In [0]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [124]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 63.45%
